## 1 - K-fold validation
#### Sources
- https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/
- https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538